#### We imported the entire necessary libraries.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib for visualization
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


##### I imported data that created in the first two question.

In [5]:
toronto_data = pd.read_csv("toronto_final.csv")

##### Use geopy library to get the latitude and longitude values of toronto.

In [6]:
address = "Toronto, TR"

geolocator = Nominatim(user_agent = "tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of toronto are 43.6561136, -79.392321.


##### I created a map of toronto with neighborhoods.

In [15]:
#create map of toronto using latitude and longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat,lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

In [18]:
central_data = toronto_data[toronto_data['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_data

,Unnamed: 0,Postal code,Borough,Neighborhood,Latitude,Longitude
0,61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
5,74,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678
6,79,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,83,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160
8,86,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049


##### I will find coordinates of 'Central Toronto'

In [20]:
addres = 'Central Toronto, CT'

geolocator = Nominatim(user_agent="ct_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude= location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6561136, -79.392321.


##### I created a map of location on "Central Toronto".

In [23]:
#create map of central toronto using latitude and longitude

map_central = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat,lng, borough in zip(central_data['Latitude'], central_data['Longitude'], central_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_central)

map_central

##### I defined Foursquare credentials

In [24]:
CLIENT_ID = 'BWQ01SGJ5CJICY0F3LSZCKC2KDUDPSQOT1TEQXPNVFZSJFMQ' # your Foursquare ID
CLIENT_SECRET = 'HCUAOVBVQSLLPDDZFTRDBUJXA20GCS0FBNQGP2LEBIDA505R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BWQ01SGJ5CJICY0F3LSZCKC2KDUDPSQOT1TEQXPNVFZSJFMQ
CLIENT_SECRET:HCUAOVBVQSLLPDDZFTRDBUJXA20GCS0FBNQGP2LEBIDA505R


##### Explore the first neighborhood in my dataframe

In [25]:
central_data.loc[0, 'Neighborhood'] #neighborhood's name

'Lawrence Park'

##### Neighborhood's latitude and longitude

In [26]:
neigh_lat = central_data.loc[0, 'Latitude']
neigh_lng = central_data.loc[0, 'Longitude']
neigh_name = central_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_lat, neigh_lng, neigh_name))

Latitude and longitude values of 43.7280205 are -79.3887901, Lawrence Park.


##### Now, let's get the top 100 venues that are in Lawrance Park  within a radius of 500 meters.

In [27]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BWQ01SGJ5CJICY0F3LSZCKC2KDUDPSQOT1TEQXPNVFZSJFMQ&client_secret=HCUAOVBVQSLLPDDZFTRDBUJXA20GCS0FBNQGP2LEBIDA505R&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [29]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5ea16c82aba297001bba8109'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
central_venues = getNearbyVenues(names=central_data['Neighborhood'],
                                   latitudes=central_data['Latitude'],
                                   longitudes=central_data['Longitude']
                                  )


Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex / North Midtown / Yorkville
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park


In [36]:
print(central_venues.shape)
central_venues.head()

(118, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Dr.Paul Hodges MIP,43.710634,-79.415810,Health & Beauty Service
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


In [37]:
central_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
Moore Park / Summerhill East,3,3,3,3,3,3
North Toronto West,23,23,23,23,23,23
Roselawn,3,3,3,3,3,3
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park,16,16,16,16,16,16
The Annex / North Midtown / Yorkville,22,22,22,22,22,22


In [38]:
print('There are {} uniques categories.'.format(len(central_venues['Venue Category'].unique())))

There are 67 uniques categories.


In [56]:

toronto_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = central_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_onehot.shape

(118, 68)

In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.0000,0.0000,0.027027,0.000000,0.027027,0.000000,0.000000,0.054054,0.000000,0.000000,0.054054,0.000000,0.027027,0.027027,0.027027,0.000000,0.081081,0.027027,0.000000,0.027027,0.000000,0.000000,0.0000,0.000000,0.027027,0.000000,0.027027,0.027027,0.054054,0.000000,0.000000,0.000000,0.000000,0.027027,0.027027,0.054054,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.027027,0.027027,0.054054,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.081081,0.027027,0.000000,0.000000,0.0000,0.0000,0.054054,0.000000,0.054054,0.027027,0.000000,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.0000,0.0000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,Moore Park / Summerhill East,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.00000

In [42]:
toronto_grouped.shape

(9, 68)

In [58]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
             venue  freq
0   Sandwich Place  0.08
1     Dessert Shop  0.08
2      Pizza Place  0.05
3  Thai Restaurant  0.05
4             Café  0.05


----Davisville North----
               venue  freq
0              Hotel  0.14
1     Sandwich Place  0.14
2               Park  0.14
3        Pizza Place  0.14
4  Food & Drink Shop  0.14


----Forest Hill North & West----
                       venue  freq
0           Sushi Restaurant  0.25
1                      Trail  0.25
2              Jewelry Store  0.25
3                       Park  0.25
4  Middle Eastern Restaurant  0.00


----Lawrence Park----
                       venue  freq
0                       Park  0.33
1                Swim School  0.33
2                   Bus Line  0.33
3        American Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Moore Park / Summerhill East----
                       venue  freq
0                 Playground  0.33
1                      Trail  0.33
2                  

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Thai Restaurant,Gym,Farmers Market
1,Davisville North,Hotel,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Pizza Place,Gas Station,Garden,Fried Chicken Joint
2,Forest Hill North & West,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
4,Moore Park / Summerhill East,Playground,Trail,Park,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Yoga Studio


In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Thai Restaurant,Gym,Farmers Market
1,Davisville North,Hotel,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Pizza Place,Gas Station,Garden,Fried Chicken Joint
2,Forest Hill North & West,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
4,Moore Park / Summerhill East,Playground,Trail,Park,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Yoga Studio


### Cluster Neighborhods

In [61]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 2, 1, 3, 1, 1])

In [62]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = central_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Unnamed: 0,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
1,62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Health & Beauty Service,Garden,Pool,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Gym,History Museum,Food & Drink Shop
2,67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Hotel,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Pizza Place,Gas Station,Garden,Fried Chicken Joint
3,68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,4,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant
4,73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Miscellaneous Shop,Park,Dessert Shop,Diner,Gift Shop,Rental Car Location


### Visualize the resulting clusters

In [63]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,-79.38879,0,Bus Line,Park,Swim School,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop


In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M4P,-79.390197,1,Hotel,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Pizza Place,Gas Station,Garden,Fried Chicken Joint
4,M4R,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Miscellaneous Shop,Park,Dessert Shop,Diner,Gift Shop,Rental Car Location
5,M5R,-79.405678,1,Café,Sandwich Place,Coffee Shop,American Restaurant,Liquor Store,Park,Pharmacy,Pizza Place,Pub,Cosmetics Shop
6,M4S,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Thai Restaurant,Gym,Farmers Market
8,M4V,-79.400049,1,Coffee Shop,Pub,American Restaurant,Restaurant,Liquor Store,Bagel Shop,Bank,Pizza Place,Fried Chicken Joint,Sushi Restaurant


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M4T,-79.38316,2,Playground,Trail,Park,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Yoga Studio


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5N,-79.416936,3,Health & Beauty Service,Garden,Pool,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Gym,History Museum,Food & Drink Shop


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5P,-79.411307,4,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant


Thank you for attention.